In [2]:
import pandas as pd

# Load the necessary datasets
precip_df = pd.read_excel('Daily_Colby_Mesonet.xlsx', usecols=['Timestamp', 'Precipitation'])
texture_df = pd.read_excel('24 KSU TAPS Soil texture.xlsx', skiprows=1)
ET_mesonet = pd.read_excel('ET_Grass_Mesonet.xlsx')
ET_mesonet = ET_mesonet.drop([0, 1], axis=0)
ET_mesonet = ET_mesonet.rename(columns={"ETo": "ETo Grass(mm)"})
ET_mesonet = ET_mesonet.rename(columns={"ETo.1": "ETo Alfalfa(mm)"})
ET_mesonet = ET_mesonet.rename(columns={"Precipitation": "Precipitation"})


Team_Plot_ID = pd.read_excel('team_plot_data.xlsx')


In [3]:
import pandas as pd

# Define the path to the Excel file
file_path_sentek = '/Users/yaniksitta/co2_dashboard/24 KSU TAPS Sentek.xlsx'

# Load the Excel file
excel_file_sentek = pd.ExcelFile(file_path_sentek)

# Initialize an empty list to hold the data from each worksheet
all_sheets_sentek = []

# Iterate over each sheet in the Excel file
for sheet_name_sentek in excel_file_sentek.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_sentek = pd.read_excel(file_path_sentek, sheet_name=sheet_name_sentek, skiprows=2)
    
    # Remove the " C" from the column names if present
    sheet_data_sentek.columns = [col.replace(" C", "")
                                     .replace("Temp Sensor #6 (55 cm)", "Temp. Sensor #6 (55 cm)")
                                 for col in sheet_data_sentek.columns]
    
    # Add a new column with the sheet name (Team Number)
    sheet_data_sentek['Team Number'] = sheet_name_sentek
    
    # Append the sheet data to the list
    all_sheets_sentek.append(sheet_data_sentek)

# Combine all sheets into a single DataFrame
combined_sentek = pd.concat(all_sheets_sentek, ignore_index=True)

# Remove all rows where column V1 value is -1
combined_sentek = combined_sentek[combined_sentek["V1"] != -1]


In [4]:
filtered_combined_sentek = combined_sentek.loc[:, ["Timestamp", "Sensor #1 (5 cm)", "Sensor #2 (15 cm)", "Sensor #3 (25 cm)", "Team Number"]]

# Round 'Timestamp' to the nearest hour
filtered_combined_sentek['Timestamp'] = pd.to_datetime(filtered_combined_sentek['Timestamp'])
filtered_combined_sentek['Rounded Timestamp'] = filtered_combined_sentek['Timestamp'].dt.round('H')

# Group by 'Rounded Timestamp' and 'Team Number', then calculate the mean for sensor columns
grouped_filtered_combined_sentek = filtered_combined_sentek.groupby(['Rounded Timestamp', 'Team Number']).agg({
    'Sensor #1 (5 cm)': 'mean',
    'Sensor #2 (15 cm)': 'mean',
    'Sensor #3 (25 cm)': 'mean'
}).reset_index()

# Calculate the weighted volumetric water content (VWC)
grouped_filtered_combined_sentek['Weighted VWC'] = (
    grouped_filtered_combined_sentek['Sensor #1 (5 cm)'] * 0.75 +
    grouped_filtered_combined_sentek['Sensor #2 (15 cm)'] * 0.15 +
    grouped_filtered_combined_sentek['Sensor #3 (25 cm)'] * 0.1
)

# Define a function to determine the time of day
def get_time_of_day(timestamp):
    hour = timestamp.hour
    if 6 <= hour < 10:
        return 'Morning'
    elif 10 <= hour < 19:
        return 'Daytime'
    else:
        return 'Night'

# Apply the function to create a 'Time of Day' column
grouped_filtered_combined_sentek['Time of Day'] = grouped_filtered_combined_sentek['Rounded Timestamp'].apply(get_time_of_day)

# Extract the date from the rounded timestamp
grouped_filtered_combined_sentek['Timestamp'] = grouped_filtered_combined_sentek['Rounded Timestamp'].dt.date

# Group by 'Date', 'Time of Day', and 'Team Number', then calculate the mean for sensor columns
grouped_by_time_of_day = grouped_filtered_combined_sentek.groupby(['Timestamp', 'Time of Day', 'Team Number']).agg({
    'Sensor #1 (5 cm)': 'mean',
    'Sensor #2 (15 cm)': 'mean',
    'Sensor #3 (25 cm)': 'mean',
    'Weighted VWC': 'mean'
}).reset_index()

# Display the resulting dataframes
grouped_filtered_combined_sentek


,Rounded Timestamp,Team Number,Sensor #1 (5 cm),Sensor #2 (15 cm),Sensor #3 (25 cm),Weighted VWC,Time of Day,Timestamp
0,2024-05-27 19:00:00,Team #3 Data,39.001410,53.949330,50.725080,42.415965,Night,2024-05-27
1,2024-05-27 20:00:00,Team #3 Data,39.579860,53.970727,50.746607,42.855165,Night,2024-05-27
2,2024-05-27 21:00:00,Team #3 Data,39.749580,53.997470,50.763830,42.988188,Night,2024-05-27
3,2024-05-27 22:00:00,Team #3 Data,39.678830,53.969385,50.760595,42.930590,Night,2024-05-27
4,2024-05-27 23:00:00,Team #3 Data,39.745160,53.977410,50.767060,42.982188,Night,2024-05-27
...,...,...,...,...,...,...,...,...
28263,2024-09-17 09:00:00,Team #23 Data,14.537970,34.299440,38.199990,19.868392,Morning,2024-09-17
28264,2024-09-17 09:00:00,Team #29 Data,13.428745,38.792965,36.077220,19.498225,Morning,2024-09-17
28265,2024-09-17 09:00:00,Team #33 Data,9.659066,30.060040,33.398015,15.093107,Morning,2024-09-17
28266,2024-09-17 09:00:00,Team #34 Data,22.175465,28.477520,35.401095,24.443336,Morning,2024-09-17


In [5]:
time_of_day_weights = {
    'Morning': 0.25,
    'Daytime': 0.6,
    'Night': 0.15
}

# Calculate the weighted average of 'Weighted VWC' for each date
weighted_VWC = grouped_by_time_of_day.groupby(['Timestamp', 'Team Number']).apply(
    lambda x: (x['Weighted VWC'] * x['Time of Day'].map(time_of_day_weights)).sum() / x['Time of Day'].map(time_of_day_weights).sum()
).reset_index(name='Weighted Average VWC')

weighted_VWC['Team Number'] = weighted_VWC['Team Number'].str.extract('(\d+)').astype(int)
# Display the resulting dataframe
weighted_VWC


,Timestamp,Team Number,Weighted Average VWC
0,2024-05-27,3,42.834419
1,2024-05-28,10,38.606774
2,2024-05-28,16,45.521697
3,2024-05-28,17,32.778285
4,2024-05-28,23,23.718222
...,...,...,...
1191,2024-09-17,23,19.882753
1192,2024-09-17,29,19.483833
1193,2024-09-17,33,15.032186
1194,2024-09-17,34,24.514431


In [6]:
texture_df = texture_df[texture_df["Sample Depth (in)"].isin(["0 - 12", "3 - 9", "4 - 9"])]
texture_df = texture_df.drop(columns=["Lat", "Lng"])


#Soil Water Capacity
#texture_df = pd.merge(texture_df, Team_Plot_ID, on='Plot ID', how='inner')
texture_df = texture_df.merge(Team_Plot_ID[['Plot ID', 'Team Number']], on='Plot ID', how='left')

texture_df

,Plot ID,Lab_ID,Sample ID,Sample Depth (in),OMC (%),Soil Textural Class,Sand (%),Silt (%),Clay (%),Team Number
0,301,119912,1,0 - 12,3.3,Silty Clay Loam,17.6,54.9,27.5,5
1,404,119922,11,0 - 12,1.7,Silt Loam,21.7,55.9,22.4,15
2,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8,16
3,1203,119942,31,0 - 12,2.0,Clay Loam,23.2,49.5,27.3,22
4,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4,10
5,1802,119962,51,0 - 12,1.8,Silt Loam,20.5,57.2,22.3,18
6,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5,10
7,2403,119982,71,0 - 12,2.0,Silt Loam,21.4,56.2,22.4,21
8,2405,119992,81,4 - 9,1.6,Silt Loam,20.5,55.9,23.6,20
9,2604,120002,91,3 - 9,1.2,Silt Loam,20.1,57.4,22.5,28


In [7]:
merged_df = pd.merge(weighted_VWC, texture_df, on='Team Number', how='inner')

merged_df

,Timestamp,Team Number,Weighted Average VWC,Plot ID,Lab_ID,Sample ID,Sample Depth (in),OMC (%),Soil Textural Class,Sand (%),Silt (%),Clay (%)
0,2024-05-28,10,38.606774,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4
1,2024-05-28,10,38.606774,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5
2,2024-05-29,10,38.408291,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4
3,2024-05-29,10,38.408291,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5
4,2024-05-30,10,39.898397,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4
...,...,...,...,...,...,...,...,...,...,...,...,...
326,2024-09-07,16,22.552820,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8
327,2024-09-08,16,22.011092,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8
328,2024-09-09,16,21.451701,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8
329,2024-09-10,16,20.826845,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8


In [8]:
ET_mesonet['Timestamp'] = pd.to_datetime(ET_mesonet['Timestamp']).dt.date

merged_df = pd.merge(merged_df, ET_mesonet, on='Timestamp', how='inner')
merged_df

,Timestamp,Team Number,Weighted Average VWC,Plot ID,Lab_ID,Sample ID,Sample Depth (in),OMC (%),Soil Textural Class,Sand (%),Silt (%),Clay (%),Station,Precipitation,ETo Grass(mm),ETo Alfalfa(mm)
0,2024-05-28,10,38.606774,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4,Colby,7.11,4.23,5.5
1,2024-05-28,10,38.606774,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5,Colby,7.11,4.23,5.5
2,2024-05-28,16,45.521697,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,53.5,24.8,Colby,7.11,4.23,5.5
3,2024-05-29,10,38.408291,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4,Colby,0,6.29,8.37
4,2024-05-29,10,38.408291,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5,Colby,0,6.29,8.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2024-09-14,10,29.636197,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5,Colby,0,4.53,6.35
327,2024-09-15,10,28.451880,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4,Colby,0.76,6.3,9.25
328,2024-09-15,10,28.451880,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,53.4,25.5,Colby,0.76,6.3,9.25
329,2024-09-16,10,27.991859,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,53.5,22.4,Colby,0.51,5.36,7.75


In [9]:
import numpy as np
merged_df['PWP (%)'] = (((0.14) + (0.0033 * merged_df['Clay (%)'])) - (0.0022 * merged_df['Sand (%)']) + (0.0006 * merged_df['OMC (%) ']))*100
merged_df['Field Capacity (%)'] = ((0.31 + (0.0017 * merged_df['Clay (%)'])) - (0.0036 * merged_df['Sand (%)']) + (0.001 * merged_df['OMC (%) ']))*100
merged_df['Field Capacity (mm)'] = ((merged_df['Field Capacity (%)']/100)*135)
merged_df['Soil Water Storage (mm)'] = (merged_df['Weighted Average VWC']/100)*135
merged_df['Soil Water Depletion'] = (merged_df['Field Capacity (mm)'] - merged_df['Soil Water Storage (mm)'])
merged_df['Adjusted SWD'] = (merged_df['Soil Water Depletion'] + merged_df['Precipitation'])
merged_df['Crop Coefficient Corn (Kc)'] = (1.15)
merged_df['Evapotranspiration Corn (ET-Corn)'] = (merged_df['ETo Grass(mm)'] * merged_df['Crop Coefficient Corn (Kc)'])
merged_df['Total Water Needed based on Corn Kc'] = (merged_df['Evapotranspiration Corn (ET-Corn)'] + merged_df['Adjusted SWD'])
merged_df['Total Water Needed based on Corn Kc'] = np.where(merged_df['Total Water Needed based on Corn Kc'] < 0, 0, merged_df['Total Water Needed based on Corn Kc'])
merged_df['Recommended Irrigation'] = merged_df['Total Water Needed based on Corn Kc'] - merged_df['Precipitation']
merged_df['Recommended Irrigation'] = np.where(merged_df['Recommended Irrigation'] < 0, 0, merged_df['Recommended Irrigation'])


merged_df

,Timestamp,Team Number,Weighted Average VWC,Plot ID,Lab_ID,Sample ID,Sample Depth (in),OMC (%),Soil Textural Class,Sand (%),...,PWP (%),Field Capacity (%),Field Capacity (mm),Soil Water Storage (mm),Soil Water Depletion,Adjusted SWD,Crop Coefficient Corn (Kc),Evapotranspiration Corn (ET-Corn),Total Water Needed based on Corn Kc,Recommended Irrigation
0,2024-05-28,10,38.606774,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,...,16.198,26.312,35.52120,52.119144,-16.597944,-9.487944,1.15,4.8645,0,0
1,2024-05-28,10,38.606774,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,...,17.869,27.899,37.66365,52.119144,-14.455494,-7.345494,1.15,4.8645,0,0
2,2024-05-28,16,45.521697,1105,119932,21,0 - 12,1.7,Silt Loam,21.7,...,17.512,27.574,37.22490,61.454291,-24.229391,-17.119391,1.15,4.8645,0,0
3,2024-05-29,10,38.408291,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,...,16.198,26.312,35.52120,51.851193,-16.329993,-16.329993,1.15,7.2335,0,0
4,2024-05-29,10,38.408291,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,...,17.869,27.899,37.66365,51.851193,-14.187543,-14.187543,1.15,7.2335,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,2024-09-14,10,29.636197,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,...,17.869,27.899,37.66365,40.008865,-2.345215,-2.345215,1.15,5.2095,2.864285,2.864285
327,2024-09-15,10,28.451880,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,...,16.198,26.312,35.52120,38.410037,-2.888837,-2.128837,1.15,7.245,5.116163,4.356163
328,2024-09-15,10,28.451880,2401,119972,61,4 - 9,1.6,Silt Loam,21.1,...,17.869,27.899,37.66365,38.410037,-0.746387,0.013613,1.15,7.245,7.258613,6.498613
329,2024-09-16,10,27.991859,1706,119952,41,0 - 12,1.8,Silt Loam,24.1,...,16.198,26.312,35.52120,37.789009,-2.267809,-1.757809,1.15,6.164,4.406191,3.896191


In [10]:
output_path = 'merged_df_TERMINADO.xlsx'
merged_df.to_excel(output_path, index=False)

In [11]:
import pandas as pd
import plotly.graph_objects as go
import panel as pn

# Enable Panel extension
pn.extension(sizing_mode='stretch_width')

# Use the merged DataFrame 'merged_df'
irrigation_data = merged_df.copy()

# Convert 'Timestamp' to datetime if it's not already
irrigation_data['Timestamp'] = pd.to_datetime(irrigation_data['Timestamp'])

# Remove duplicate timestamps for the same irrigation recommendation from irrigation_data
irrigation_data = irrigation_data.sort_values('Timestamp').drop_duplicates(subset=['Timestamp'], keep='first')

# Create a list of unique team numbers
team_numbers = irrigation_data['Team Number'].unique()

# Create a dropdown menu for Team Numbers
team_selector = pn.widgets.Select(name='Select Team Number', options=team_numbers.tolist(), value=team_numbers[0])

# Create a date range slider for selecting timestamps
date_range_slider = pn.widgets.DateRangeSlider(name='Date Range', 
                                               start=irrigation_data['Timestamp'].min(), 
                                               end=irrigation_data['Timestamp'].max(), 
                                               value=(irrigation_data['Timestamp'].min(), irrigation_data['Timestamp'].max()))

# Function to update the graph based on the selected team and date range
def update_graph(selected_team, date_range):
    # Create an empty figure
    fig = go.Figure()

    # Filter data for the selected team and date range
    filtered_data = irrigation_data[
        (irrigation_data['Team Number'] == selected_team) &
        (irrigation_data['Timestamp'] >= date_range[0]) &
        (irrigation_data['Timestamp'] <= date_range[1])
    ]

    # Add traces for Precipitation and Recommended Irrigation
    fig.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Precipitation'],
                             mode='lines', name='Precipitation'))
    fig.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Recommended Irrigation'],
                             mode='lines', name='Recommended Irrigation'))

    # Update the layout for the entire chart
    fig.update_layout(
        title=f"Precipitation and Recommended Irrigation for Team {selected_team} Over Time",
        xaxis_title="Timestamp",
        yaxis_title="Water Depletion in mm",
        hovermode="x unified"
    )

    return fig

# Panel plot
@pn.depends(team_selector.param.value, date_range_slider.param.value)
def plot(selected_team, date_range):
    return pn.pane.Plotly(update_graph(selected_team, date_range))

# Layout the dashboard
dashboard_irrigation = pn.Column(
    pn.pane.Markdown("# Irrigation and Precipitation Dashboard", width=1400),
    team_selector,
    date_range_slider,
    plot,
    sizing_mode='fixed'
)



# Show the dashboard
dashboard_irrigation.servable()



pn.extension('plotly')



Column(sizing_mode='fixed')
    [0] Markdown(str, width=1400)
    [1] Select(options=[10, 16], sizing_mode='stretch_width', value=10)
    [2] DateRangeSlider(end=Timestamp('2024-09-16 0..., name='Date Range', sizing_mode='stretch_width', start=Timestamp('2024-05-28 0..., value=(Timestamp('2024-05-28 00:..., value_end=Timestamp('2024-09-16 0..., value_start=Timestamp('2024-05-28 0...)
    [3] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')